In [6]:
# MAIN code for prediction
# method: CNN-BiLSTM
# requirements: python3.7 keras2.2.5 tensorflow-cpu1.15

In [9]:
#------------------------------------------------------------------------------
# IMPORTS
from keras.layers import Input, Dense, LSTM ,Conv1D,Dropout,Bidirectional,Multiply
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.utils.vis_utils import plot_model
from numpy.random import seed
from keras.models import Model
from keras.layers.core import *
from keras.models import *
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import vstack,hstack

In [10]:
#------------------------------------------------------------------------------
# FUNCTIONS
#--CNN1D
def train_cnn(x_tra,y_tra,temp_save_path,iens):
    
    #set early stopping criteria
    pat = 20 #this is the number of epochs with no improvment after which the training will stop
    early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=0)
   
    #define the model checkpoint callback -> this will keep on saving the model as a physical file
    model_save_name = '{0}cnn_model_ens{1:0>3}.h5'.format(*temp_save_path,iens)
    model_checkpoint = ModelCheckpoint(model_save_name, verbose=0, save_best_only=True)
   
    n_folds=5
    #save the model history in a list after fitting so that we can plot later
    model_history = []
    for i in range(n_folds):
        print("Training on Fold: ",i+1)
        model = my_cnn_model([x_tra.shape[1], 1])
        history=model.fit(x_tra, y_tra,epochs=1000, callbacks=[early_stopping,model_checkpoint], 
                 batch_size=16, shuffle=True, verbose=1, validation_split=0.1)
        model_history.append(history)
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        # epochs = range(len(loss))
        # plt.figure()
        # plt.plot(epochs, loss, 'r', label='Training loss')
        # plt.plot(epochs, val_loss, 'b', label='validation loss')
        # plt.title('Training and validation loss')
        # plt.legend()
        np.savez("{0}traineval_ens{1:0>3}_fold{2:0>3}.npz".format(*temp_save_path,iens,i), loss=loss,val_loss=val_loss)
       
    model = load_model(model_save_name)
    return model

def my_cnn_model(input_size):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=1, activation='relu', 
                     input_shape=input_size,kernel_initializer="he_uniform"))
    # model.add(Dropout(0.5))
    # model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
    # model.add(Dropout(0.5))
    
    model.add(Flatten())
    # model.add(Dense(64, activation='relu'))
    # model.add(Dense(64, activation='relu'))
    model.add(Dense(1,activation='linear'))
    # model.compile(optimizer='adam', loss='mse')
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.9, 
                                            beta_2=0.999, epsilon=1e-07, 
                                            amsgrad=False,decay=1e-06), loss='mse')
    # model.compile(optimizer=optimizers.SGD(learning_rate=0.1, momentum=0.1), loss='mse')
    # model.compile(optimizer=optimizers.RMSprop(learning_rate=0.01),loss='mse')
    return model

#--BiLSTM
def attention_model(window, dims,lunits):
    inputs = Input(shape=(window, dims))

    x = Conv1D(filters = 32, kernel_size = 1, activation = 'relu',kernel_initializer="he_uniform")(inputs)  #, padding = 'same'
    # x = Dropout(0.7)(x)
    #lstm_out = Bidirectional(LSTM(lstm_units, activation='relu'), name='bilstm')(x)
    lstm_out = Bidirectional(LSTM(lunits, return_sequences=True))(x)
    lstm_out = Dropout(0.5)(lstm_out)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    
    output = Dense(1, activation='linear')(attention_mul)
    model = Model(inputs=[inputs], outputs=output)
    return model

SINGLE_ATTENTION_VECTOR = False
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = inputs
    #a = Permute((2, 1))(inputs)
    #a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(input_dim, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((1, 2), name='attention_vec')(a)

    #output_attention_mul = concatenate([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

#--others
def create_dataset(dataset, look_back):

    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back),:]
        dataX.append(a)
        dataY.append(dataset[i + look_back,:])
    TrainX = np.array(dataX)
    Train_Y = np.array(dataY)

    return TrainX, Train_Y

In [16]:
#------------------------------------------------------------------------------
# MAIN

#--define parameters for traning
INPUT_DIMS = 1
TIME_STEPS = 10
lstm_units = 32 #16

yrs        = np.arange(1500,1949,step=1)
yrf        = np.arange(1950,2100,step=1)
nens       = 500 # repeat how many times

trend_r2   = np.zeros((nens,2)) #train,test
osc_r2     = np.zeros((nens,2)) #train.test
trend_mse  = np.zeros((nens,2)) #train,test
osc_mse    = np.zeros((nens,2)) #train.test
recent_r2  = np.zeros((nens,2)) #1960-2000 2004-2017

temp_save_path = ["./temp_cnn/"]
pdo_save_path  = ["./pdo_cnn_lstm/"]

#--read in files
# historical-model training
df_1       = pd.read_csv('historical_imfs.csv',header=None)
# df_1       = pd.read_csv('G:/Project2/final-codes/CEEMDAN_V00/hist_imf_hail.txt',header=None)
data_1     = df_1.values
data_1     = data_1.transpose() # nyear*nvar
YY_osc     = data_1[:,0] # target-oscillation
YY_trend   = data_1[:,1] # target-trend 
XX_osc     = data_1[:,2] # features-oscillation
XX_trend   = data_1[:,3] # features-trend

# future-prediction
df_2       = pd.read_csv('future_imfs.txt',header=None)
data_2     = df_2.values
data_2     = data_2.transpose()
XX_osc_fut = data_2[:,0]
XX_trend_fut=data_2[:,1:5] # ssp126,245,370,585

In [18]:
#--preprocessing

# training
Scaler_Input_trend        = MinMaxScaler(feature_range = (-1, 1) )
Input_Train_Scaler_trend  = Scaler_Input_trend.fit_transform(XX_trend.reshape(-1,1))

Scaler_Output_trend       = MinMaxScaler(feature_range = (-1, 1) ) 
Output_Train_Scaler_trend = Scaler_Output_trend.fit_transform(YY_trend.reshape(-1,1))

Scaler_Input_osc          = MinMaxScaler(feature_range = (-1, 1) )
Input_Train_Scaler_osc    = Scaler_Input_osc.fit_transform(XX_osc.reshape(-1,1))

Scaler_Output_osc         = MinMaxScaler(feature_range = (-1, 1) ) 
Output_Train_Scaler_osc   = Scaler_Output_osc.fit_transform(YY_osc.reshape(-1,1))

# forecast
Input_Forecast126_Scaler  = Scaler_Input_trend.transform(XX_trend_fut[:,0].reshape(-1,1))
Input_Forecast245_Scaler  = Scaler_Input_trend.transform(XX_trend_fut[:,1].reshape(-1,1))
Input_Forecast370_Scaler  = Scaler_Input_trend.transform(XX_trend_fut[:,2].reshape(-1,1))
Input_Forecast585_Scaler  = Scaler_Input_trend.transform(XX_trend_fut[:,3].reshape(-1,1))

Input_Forecast_Scaler     = Scaler_Input_osc.transform(XX_osc_fut.reshape(-1,1))

XX1, _    = create_dataset(Input_Train_Scaler_osc, TIME_STEPS)
_,YY1     = create_dataset(Output_Train_Scaler_osc, TIME_STEPS)
XX1_fut,_ = create_dataset(Input_Forecast_Scaler, TIME_STEPS)

(449, 5)

In [ ]:
#--train and prediction
np.random.seed(2023)
for iens in range(nens):
    
    # training for oscillation##################################
    # train
    train_X1, test_X1, train_Y1, test_Y1=train_test_split(XX1, YY1, test_size=0.2)
    m1 = attention_model(TIME_STEPS,INPUT_DIMS,lstm_units)
    m1.summary()
    m1.compile(optimizer=optimizers.Adam(learning_rate=0.001, 
                                         beta_1=0.9, beta_2=0.999, 
                                         epsilon=1e-07, amsgrad=False,decay=1e-06), loss='mse')
    model_save_name = '{0}lstm_model_ens{1:0>3}.h5'.format(*pdo_save_path,iens)
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, verbose=0),
        ModelCheckpoint(model_save_name, monitor='val_loss', save_best_only=True, verbose=0,write_grads=True),
    ]
    history  = m1.fit([train_X1], train_Y1, epochs=1000, batch_size=8, shuffle=True, verbose=1,
                      validation_split=0.1, callbacks=callbacks)
    
    loss     = history.history['loss']
    val_loss = history.history['val_loss']
    epochs   = range(len(loss))
    np.savez("{0}traineval_ens{1:0>3}.npz".format(*pdo_save_path,iens), loss=loss,val_loss=val_loss)
    
    train_Y1_pred = m1.predict(train_X1)
    test_Y1_pred  = m1.predict(test_X1)
    # evaluation
    osc_r2[iens,0]  = r2_score(train_Y1, train_Y1_pred)
    osc_r2[iens,1]  = r2_score(test_Y1,  test_Y1_pred)
    osc_mse[iens,0] = mean_squared_error(train_Y1, train_Y1_pred)
    osc_mse[iens,1] = mean_squared_error(test_Y1,  test_Y1_pred)
    # prediction
    fut_Y_pred      = m1.predict(XX1_fut)
    
    # training for trend########################################
    # train
    train_X2, test_X2, train_Y2, test_Y2=train_test_split(Input_Train_Scaler_trend, Output_Train_Scaler_trend, test_size=0.2)
    m2            = train_cnn(train_X2,train_Y2,temp_save_path,iens) 
    train_Y2_pred = m2.predict(train_X2)
    test_Y2_pred  = m2.predict(test_X2)
    # evaluation
    trend_r2[iens,0]  = r2_score(train_Y2, train_Y2_pred)
    trend_r2[iens,1]  = r2_score(test_Y2,  test_Y2_pred)
    trend_mse[iens,0] = mean_squared_error(train_Y2, train_Y2_pred)
    trend_mse[iens,1] = mean_squared_error(test_Y2,  test_Y2_pred)
    # prediction
    fut126_Y_pred = m2.predict(Input_Forecast126_Scaler)
    fut245_Y_pred = m2.predict(Input_Forecast245_Scaler)
    fut370_Y_pred = m2.predict(Input_Forecast370_Scaler)
    fut585_Y_pred = m2.predict(Input_Forecast585_Scaler)
    
    ################################################################
    # save output
    save_train  = vstack((yrf[TIME_STEPS:-1],fut_Y_pred[:,0],
                          fut126_Y_pred[TIME_STEPS:-1,0],
                          fut245_Y_pred[TIME_STEPS:-1,0],
                          fut370_Y_pred[TIME_STEPS:-1,0],
                          fut585_Y_pred[TIME_STEPS:-1,0]))
    save_train = save_train.transpose()
    sf = pd.DataFrame(save_train)
    sf.to_csv("./cnn_lstm/model_enspred{0:0>3}_scalar.csv".format(iens))